# Energies analysis viewer

In [1]:
import json
import os

from model_workflow.utils.structures import Structure, Atom

import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import nglview as nv
import pytraj as pt # Pytraj is used only to help NGL visualize multiple pdbs at once

Set a function to load an analysis json file

In [2]:
# Set a function to read and mine energies data
def get_energies_data (filename : str) -> dict:
    
    # Read the analysis json file
    with open(filename, 'r') as file:
        parsed_response = json.loads(file.read())
    analysis_data = parsed_response['data']

    # In this case, there is only one defined interaction: the protein-ligand interaction
    interaction_data = analysis_data[0]

    # In this case, the protein is the agent 1 and the ligand is the agent 2
    # We get the protein data
    protein_data = interaction_data['agent1']

    # For this example, we are going to focus in the overall sum of energies
    energies_data = protein_data['both']
    
    return energies_data

Hardcode a few things in order to better represent the graphs

Get the wild type absolute energies graphs

In [3]:
analyses = { 'reference': 'ref.energies.json', 'test': 'md.energies.json' }
data = []

# Load each analysis: the reference and the test
for title, filename in analyses.items():
    # Set the axis values
    y = get_energies_data(filename)
    x = [ i for i, v in enumerate(y) ]
    # Set the plot data
    data.append(
        go.Scatter(
            x = x,
            y = y,
            name = title,
        )
    )

# Set the figure
fig = go.Figure({
    "data": data,
    "layout": go.Layout(
        title='Energies',
        xaxis={
            'title': {
                'text': 'Residues',
            },
        },
        yaxis={
            'title': {
                'text': 'Electrostatic + VdW energy (kcal/mol)',
            },
        }
    )
})

# Show the figure
fig.show()

Load a sample obtained from the workflow

In [4]:
# Load the sample
sample_filename = 'energies-sample.json'
with open(sample_filename, 'r') as file:
    sample = json.loads(file.read())
    
# Mine sample data
agent1_energies = sample['agent1']['energies']
agent1_pdb = sample['agent1']['pdb']
agent2_energies = sample['agent2']['energies']
agent2_pdb = sample['agent2']['pdb']
box_origin = sample['box']['origin']
box_size = sample['box']['size']

Build the map color

In [5]:
import matplotlib

# Red-White-Blue color scale
cmap = matplotlib.cm.get_cmap('RdBu')

# Save each atom color according to its energy value
atoms_by_energy = {}
agent2_atom_indices_offset = len(agent1_energies)
for a, agent_energies in enumerate([ agent1_energies, agent2_energies ]):

    # Get specific energies (electrostatic / van der waals / both) for each atom
    specific_energies = [ atom_energies[2] for atom_energies in agent_energies ]

    # Get the minimum, maximum and range of energy values
    max_specific_energy = max(specific_energies)
    min_specific_energy = min(specific_energies)
    specific_energies_range = max_specific_energy - min_specific_energy

    # Save each atom color according to its energy value
    # Each color is saved next to its stringified atom index
    for i, specific_energy in enumerate(specific_energies):
        # Set the index of the atom
        # Indices for the agent 2 are offseted
        index = i + agent2_atom_indices_offset if a == 1 else i
        # Those energies which are pure 0s are painted black
        if specific_energy == 0:
            # Values set to absolute 0 are values which returned super high values in CMIP
            color = 'black'
        else:
            # Normalize the energy value
            normalized = (specific_energy - min_specific_energy) / specific_energies_range
            color = matplotlib.colors.to_hex(cmap(normalized))
        # Save the index of the current atom in its corresponding color
        current_atoms = atoms_by_energy.get(color, None)
        if current_atoms == None:
            atoms_by_energy[color] = [index]
        else:
            current_atoms.append(index)

# Note that doing the color map this way instead of adding one list element per atom is way more efficient
color_map = [ [color, '@'+','.join([ str(i) for i in atom_indices ])] for color, atom_indices in atoms_by_energy.items() ]
# Build the NGL color scheme
color_scheme = 'energies'
nv.color.ColormakerRegistry.add_selection_scheme(color_scheme, color_map)

Check the number of atoms in the map and check it matches the number of atoms in the structure

In [6]:
# Count the number of energy values
color_map_natoms = len(agent1_energies + agent2_energies)
print('Number of atoms in the color map: ' + str(color_map_natoms))

# Merge the structure of each interacting agent and count the number of atoms of both structures together
agent1_structure = Structure.from_pdb_file(agent1_pdb)
agent2_structure = Structure.from_pdb_file(agent2_pdb)
structure = agent1_structure.merge(agent2_structure)
structure_natoms = len(structure.atoms)
print('Number of atoms in the structure: ' + str(structure_natoms))

Number of atoms in the color map: 12584
Number of atoms in the structure: 12584


Add a few "fake" atoms to the structure to represent the CMIP grid box in NGL

In [7]:
print('Box origin: ' + str(box_origin))
print('Box size: ' + str(box_size))

# Set the positions of the vertices (corners) of the grid box
vertex_positions = []
vertex_positions.append((box_origin[0], box_origin[1], box_origin[2]))
vertex_positions.append((box_origin[0] + box_size[0], box_origin[1], box_origin[2]))
vertex_positions.append((box_origin[0], box_origin[1] + box_size[1], box_origin[2]))
vertex_positions.append((box_origin[0], box_origin[1], box_origin[2] + box_size[2]))
vertex_positions.append((box_origin[0] + box_size[0], box_origin[1] + box_size[1], box_origin[2]))
vertex_positions.append((box_origin[0] + box_size[0], box_origin[1], box_origin[2] + box_size[2]))
vertex_positions.append((box_origin[0], box_origin[1] + box_size[1], box_origin[2] + box_size[2]))
vertex_positions.append((box_origin[0] + box_size[0], box_origin[1] + box_size[1], box_origin[2] + box_size[2]))

# Add an atom for each vertex
for v, vertex_position in enumerate(vertex_positions):
    vertex_atom = Atom('VX' + str(v), 'Zn', vertex_position)
    structure.set_new_atom(vertex_atom)
    
# Set the pairs of atoms to be connected by the box edges
atomPair = [
     [ "0:X.VX0", "0:X.VX1" ],
     [ "0:X.VX0", "0:X.VX2" ],
     [ "0:X.VX0", "0:X.VX3" ],
   
     [ "0:X.VX1", "0:X.VX4" ],
     [ "0:X.VX1", "0:X.VX5" ],
   
     [ "0:X.VX2", "0:X.VX4" ],
     [ "0:X.VX2", "0:X.VX6" ],
   
     [ "0:X.VX3", "0:X.VX5" ],
     [ "0:X.VX3", "0:X.VX6" ],
   
     [ "0:X.VX4", "0:X.VX7" ],
     [ "0:X.VX5", "0:X.VX7" ],
     [ "0:X.VX6", "0:X.VX7" ]
]

Box origin: [39.83, 28.015, 46.175]
Box size: [48.5, 46.5, 30.5]


Represent the structure with the previous color scheme

In [8]:
# Load the PDB file in NGL
sample_pdb = 'energies/sample.pdb'
structure.generate_pdb_file(sample_pdb)
view = nv.show_structure_file(sample_pdb)
view.clear_representations()

# Set a representation for the main structure
view.add_representation(
    repr_type='ball+stick',
    color=color_scheme,
)

# Set a representation for the CMIP grid box
view.add_representation(
    repr_type='distance',
    atomPair= atomPair,
    labelColor= 'transparent',
    color= 'black'
)

view

NGLWidget()